In [2]:
import pandas as pd  
import yfinance as yf
import stock_pricing as sp
     
# making data frame  
df = pd.read_csv("stockerbot-export.csv", on_bad_lines='skip')  
   
df = df.iloc[:, :8]
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%a %b %d %H:%M:%S +0000 %Y', errors='coerce')
df = df.dropna(subset=['timestamp'])

# Access date components for each timestamp
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['year'] = df['timestamp'].dt.year

df['Price Day Before Tweet'] = pd.NA
df['Price Day of Tweet'] = pd.NA
df['Price Day After Tweet'] = pd.NA

In [3]:
df.head(5)

,id,text,timestamp,source,symbols,company_names,url,verified,month,day,year,Price Day Before Tweet,Price Day of Tweet,Price Day After Tweet
0,1019696670777503700,VIDEO: “I was in my office. I was minding my o...,2018-07-18 21:33:26,GoldmanSachs,GS,The Goldman Sachs,https://twitter.com/i/web/status/1019696670777...,True,7,18,2018,<NA>,<NA>,<NA>
1,1019709091038548000,The price of lumber $LB_F is down 22% since hi...,2018-07-18 22:22:47,StockTwits,M,Macy's,https://twitter.com/i/web/status/1019709091038...,True,7,18,2018,<NA>,<NA>,<NA>
2,1019711413798035500,Who says the American Dream is dead? https://t...,2018-07-18 22:32:01,TheStreet,AIG,American,https://buff.ly/2L3kmc4,True,7,18,2018,<NA>,<NA>,<NA>
3,1019716662587740200,Barry Silbert is extremely optimistic on bitco...,2018-07-18 22:52:52,MarketWatch,BTC,Bitcoin,https://twitter.com/i/web/status/1019716662587...,True,7,18,2018,<NA>,<NA>,<NA>
4,1019718460287389700,How satellites avoid attacks and space junk wh...,2018-07-18 23:00:01,Forbes,ORCL,Oracle,http://on.forbes.com/6013DqDDU,True,7,18,2018,<NA>,<NA>,<NA>


In [4]:
pd.set_option('display.max_rows', None)
ticker_lst = df['symbols'].unique().tolist()
print(len(ticker_lst))

453


In [5]:
stock = yf.Ticker('M')
stock_df = stock.history(period="10y")

In [6]:
import stock_pricing as sp

hi = sp.create_df(ticker_lst)

FB-GOOGL-GOOG: No data found, symbol may be delisted
HRS: No data found, symbol may be delisted
INTC-USB: No data found, symbol may be delisted
AMZN-GPS: No data found, symbol may be delisted
TICKER: No data found, symbol may be delisted
CBS: No data found, symbol may be delisted
TWTR: No data found, symbol may be delisted
FB: No data found, symbol may be delisted
INFO: No data found, symbol may be delisted
JEC: No data found, symbol may be delisted
FNSR: No data found, symbol may be delisted
RE: No data found, symbol may be delisted
TMK: No data found, symbol may be delisted
NUAN: No data found, symbol may be delisted
TRQ: No data found, symbol may be delisted
DCIX: No data found, symbol may be delisted
FBHS: No data found, symbol may be delisted
PAH: No price data found, symbol may be delisted (period=10y)
WYN: No price data found, symbol may be delisted (period=10y)
LUK: No price data found, symbol may be delisted (period=10y)
FMSA: No price data found, symbol may be delisted (perio

In [7]:
result_df = sp.add_price_data(hi, df, ticker_lst)

In [ ]:
result_df.head(20)

In [ ]:
result_df[result_df['symbols'] == 'UDR']

In [ ]:
hi['UDR']